In [14]:
import numpy as np 
import pandas as pd 
import portion as P
from scipy.optimize import linprog
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable 

# Transforming LCL:s to linear problems

Goal is to find $\alpha, \beta, a,b,c...$ such that if some $A, A, B$ is possible for active/passive nodes, then the corresponding $a+a+b \leq \beta / \geq \alpha$. Additionally it would be great to see if these values $a, b, c...$ could be expanded to intervals.

Example: problem

```
A AB AB

B AB AB
```
can be interpreted as $\alpha = 1, \beta=2, A = \left[ 0,\frac{1}{3}\right), B =  \left(\frac{2}{3}, 1\right]$.

What is the system of linear inequalities that could imply this?

\begin{align*}
3a &\leq \beta \\
2a+b &\leq \beta \\
a+2b &\leq \beta \\
3b &> \beta \\
\\
3a &< \alpha \\
2a+b &\geq \alpha \\
a+2b &\geq \alpha \\
3b &\geq \alpha

\end{align*}

In [15]:
epsilon = 0.0001

model = LpProblem(name="Reductions", sense=LpMaximize)

a = LpVariable(name = "a", lowBound=0, upBound=1)
b = LpVariable(name = "b", lowBound=0, upBound=1)
alpha = LpVariable(name = "alpha", lowBound=0, upBound=10)
beta = LpVariable(name = "beta", lowBound=0, upBound=10)
    
model += b-a

model += (3*a<=beta, "1")
model += (2*a+b<=beta, "2")
model += (a+2*b<=beta, "3")
model += (3*b>=beta+epsilon, "4")

model += (3*a<=alpha-epsilon, "5")
model += (2*a+b>=alpha, "6")
model += (a+2*b>=alpha, "7")
model += (a+2*b>=alpha, "8")


print("\n", model, "\n")

# Search for solutions to the given system of inequalities
if model.solve() == 1:
    print("Success!")

for var in model.variables():
    print(var, var.value())


 Reductions:
MAXIMIZE
-1*a + 1*b + 0
SUBJECT TO
1: 3 a - beta <= 0

2: 2 a + b - beta <= 0

3: a + 2 b - beta <= 0

4: 3 b - beta >= 0.0001

5: 3 a - alpha <= -0.0001

6: 2 a - alpha + b >= 0

7: a - alpha + 2 b >= 0

8: a - alpha + 2 b >= 0

VARIABLES
a <= 1 Continuous
alpha <= 10 Continuous
b <= 1 Continuous
beta <= 10 Continuous
 

Success!
alpha 0.0001
b 1.0
beta 2.0
